In [10]:
import arcpy
from arcpy import *
from arcpy.sa import *
from arcpy.ia import *
import os
import gc
from datetime import datetime

areas = ['DENTONHILLS', 'VICTORIA', 'WRIGHT', 'TAYLOR']
bio_attributes = ['CYANO', 'MOSS', 'LICHEN', 'SPRINGTAILS', 'MITES']
bio_attributes = ['MOSS', 'LICHEN', 'SPRINGTAILS', 'MITES']

arcpy.env.workspace       = 'D:\\PROJECT_ANTARCTICA\\antarctica\\antarctica_arcgis\\antarctica_arcgis.gdb\\'
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

print('Started at ' +  datetime.now().strftime("%H:%M:%S"))
                                            
dem = Raster('icefree_dem')
cti = Raster('icefree_cti')
slope = Raster('icefree_slope')
aspect = Raster('icefree_aspect')
rel_aspect = Raster('icefree_rel_aspect')
rr100 = Raster('icefree_rr100')
rr1000 = Raster('icefree_rr1000')
coast_dis = Raster('icefree_coastdist')
edge_dist = Raster('icefree_edgedist')


folder = 'D:\\PROJECT_ANTARCTICA\\antarctica\\OUTPUT\\'

## FBCR train and predict
for bio in bio_attributes:

    prediction_type = "PREDICT_RASTER"
    in_features = "NZTAB_SURVEYPOINTS"
    variable_predict = "CAT_"+bio
    treat_variable_as_categorical = "CATEGORICAL" 
    explanatory_variables = None
    distance_features = None
    explanatory_rasters = [[dem, "false"], [cti, "false"], [slope, "false"], [aspect, "false"], [rel_aspect, "false"], [rr100, "false"], [rr1000, "false"], [coast_dis, "false"], [edge_dist, "false"]]
    features_to_predict = None
    output_features = None
    output_raster = r'temp_raster'
   
    percentage_for_training = 30
    
    explanatory_variable_matching = None
    explanatory_distance_matching = None
    explanatory_rasters_matching = [[dem, "dem"], [cti, "cti"], [slope, "slope"], [aspect, "aspect"], [rel_aspect, "rel_aspect"], [rr100, "rr100"], [rr1000, "rr1000"], [coast_dis, "coast_dis"], [edge_dist, "edge_dist"]]
    output_trained_features = None
    use_raster_values = True
    number_of_trees = 300
    minimum_leaf_size = 30
    maximum_level = None
    sample_size = 100
    random_sample = None
    output_validation_table = None
    compensate_sparse_categories = None
    number_validation_runs = None
    calculate_uncertainty = None
    
    output_importance_table = folder + "FBCR_VI_"+ bio+"_CAT_"+str(percentage_for_training)+".dbf"
    output_classification_table = folder + "CONMATRIX_"+ bio+"_CAT_"+str(percentage_for_training)+".dbf"
    output_trained_model =  folder + "TRAINEDMODEL_"+ bio+"_CAT_"+str(percentage_for_training)
    
    arcpy.stats.Forest(prediction_type, in_features, variable_predict,
                        treat_variable_as_categorical, explanatory_variables, distance_features,
                        explanatory_rasters, features_to_predict, output_features, output_raster,
                        explanatory_variable_matching, explanatory_distance_matching, 
                        explanatory_rasters_matching, output_trained_features, output_importance_table,
                        use_raster_values, number_of_trees, minimum_leaf_size, maximum_level,
                        sample_size, random_sample, percentage_for_training,
                        output_classification_table, output_validation_table, compensate_sparse_categories, number_validation_runs, calculate_uncertainty, output_trained_model)
    
    output = Raster("temp_raster")
    output.save(folder + "FBCR_PREDICTED_"+ bio+"_CAT_"+str(percentage_for_training)+".tif")
    print(bio + ' presence prediction finished at ' +  datetime.now().strftime("%H:%M:%S"))
    
    
    prediction_type = "PREDICT_RASTER"
    in_features = "NZTAB_SURVEYPOINTS"
    variable_predict = "COV_"+bio
    treat_variable_as_categorical = "NUMERIC" 
    explanatory_variables = None
    distance_features = None
    explanatory_rasters = [[dem, "false"], [cti, "false"], [slope, "false"], [aspect, "false"], [rel_aspect, "false"], [rr100, "false"], [rr1000, "false"], [coast_dis, "false"], [edge_dist, "false"]]
    features_to_predict = None
    output_features = None
    output_raster = r'temp_raster'

    percentage_for_training = 30
    
    explanatory_variable_matching = None
    explanatory_distance_matching = None
    explanatory_rasters_matching = [[dem, "dem"], [cti, "cti"], [slope, "slope"], [aspect, "aspect"], [rel_aspect, "rel_aspect"], [rr100, "rr100"], [rr1000, "rr1000"], [coast_dis, "coast_dis"], [edge_dist, "edge_dist"]]
    output_trained_features = None
    use_raster_values = True
    number_of_trees = 300
    minimum_leaf_size = 30
    maximum_level = None
    sample_size = 100
    random_sample = None
    output_classification_table = None
    compensate_sparse_categories = None
    number_validation_runs = 2
    calculate_uncertainty = None
    
    output_importance_table = folder + "FBCR_VI_"+ bio+"_NUM_"+str(percentage_for_training)+".dbf"
    output_validation_table = folder + "RSQUARE_"+ bio+"_NUM_"+str(percentage_for_training)+".dbf"
    output_trained_model =  folder + "TRAINEDMODEL_"+ bio+"_NUM_"+str(percentage_for_training)

    arcpy.stats.Forest(prediction_type, in_features, variable_predict,
                        treat_variable_as_categorical, explanatory_variables, distance_features,
                        explanatory_rasters, features_to_predict, output_features, output_raster,
                        explanatory_variable_matching, explanatory_distance_matching, 
                        explanatory_rasters_matching, output_trained_features, output_importance_table,
                        use_raster_values, number_of_trees, minimum_leaf_size, maximum_level,
                        sample_size, random_sample, percentage_for_training,
                        output_classification_table, output_validation_table, compensate_sparse_categories, number_validation_runs, calculate_uncertainty, output_trained_model)

    output = Raster("temp_raster")
    output.save(folder + "FBCR_PREDICTED_"+ bio+"_NUM_"+str(percentage_for_training)+".tif")
    print(bio + ' coverage prediction finished at ' +  datetime.now().strftime("%H:%M:%S"))
    
print('Ended at ' +  datetime.now().strftime("%H:%M:%S"))
del(output_raster, output)
gc.collect()

Started at 10:09:52
MOSS presence prediction finished at 10:12:12
MOSS coverage prediction finished at 10:13:55
LICHEN presence prediction finished at 10:16:38
LICHEN coverage prediction finished at 10:18:21
SPRINGTAILS presence prediction finished at 10:20:51
SPRINGTAILS coverage prediction finished at 10:22:37
MITES presence prediction finished at 10:24:59
MITES coverage prediction finished at 10:26:41
Ended at 10:26:41


459